# Imports

In [1]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

In [2]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image

# Constants

In [3]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/' 

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUT = IMAGE_HEIGHT*IMAGE_WIDTH*CHANNELS

# Get the Data

In [4]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [7]:

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

In [8]:


x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

# Explore

In [9]:
x_train_all.shape

(60000, 784)

In [10]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [11]:
y_train_all.shape

(60000,)

In [12]:
x_test.shape

(10000, 784)

In [13]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [14]:
# Re-scale
x_train_all, x_test = x_train_all/255, x_test/255

#### convert target values to one hot encoding

In [15]:
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [16]:
np.eye(10)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [17]:
np.eye(10)[2]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
np.eye(10)[[2,0]]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [19]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [20]:
y_train_all.shape

(60000, 10)

In [21]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Create validation dataset from training data

In [22]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [23]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [24]:
x_train.shape

(50000, 784)

In [25]:
x_val.shape

(10000, 784)

# Setup Tensorflow Graph

In [26]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUT], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

### Neural Networks Architecture
#### Hyperparameters

In [27]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [28]:
def setup_layer(input, weight_dim, bias_dim, name):
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name == 'out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
        return layer_out

In [29]:
# model without dropout
# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUT, n_hidden1],
#                       bias_dim=[n_hidden1], name='layer_1')

# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2],
#                       bias_dim=[n_hidden2], name='layer_2')

# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
#                       bias_dim=[NR_CLASSES], name='out')

# model_name = f'{n_hidden1} - {n_hidden2} LR{learning_rate} E{nr_epochs}'

In [30]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUT, n_hidden1],
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')


layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2],
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1} -DO- {n_hidden2} LR{learning_rate} E{nr_epochs}'

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [31]:
# folder for tensorboard

folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories!')

Successfully created directories!


# Loss, Optimization and Metrics

#### Defining loss function

In [32]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining Optimizer

In [33]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Acuracy Metric

In [34]:
with tf.name_scope('accuracy_calc'):
    model_prediction = tf.argmax(output, axis=1, name='prediction')
    correct_pred = tf.equal(model_prediction, tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [35]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [36]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [37]:
sess = tf.Session()

In [38]:
directory + '/train'

'tensorboard_mnist_digit_logs/512 -DO- 64 LR0.001 E50 at 08 55/train'

#### setup filewirter and merge summaries

In [39]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

#### initialize all the varibales

In [40]:
init = tf.global_variables_initializer()
sess.run(init)

## Batching the data

In [41]:
size_of_batch = 1000

In [42]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples / size_of_batch)

index_in_epoch = 0

In [43]:
nr_iterations

50

In [44]:
def next_batch(batch_size, data, labels):

    global num_examples
    global index_in_epoch

    start = index_in_epoch
    index_in_epoch += batch_size

    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size

    end = index_in_epoch

    return data[start:end], labels[start:end]

### Training Loop

In [45]:
for epoch in range(nr_epochs):
    
    # ************** Training Dataset *******
    for i in range(nr_iterations):

        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)

        feed_dictionary = {X:batch_x, Y:batch_y}

        sess.run(train_step, feed_dict=feed_dictionary)

    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)

    train_writer.add_summary(s, epoch)

    print(f'Epoch {epoch} \t| Training Accuracy {batch_accuracy}')

    # ************** Validation *****************
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)
    
print('Done Training!')

Epoch 0 	| Training Accuracy 0.8700000047683716
Epoch 1 	| Training Accuracy 0.8799999952316284
Epoch 2 	| Training Accuracy 0.8840000033378601
Epoch 3 	| Training Accuracy 0.9549999833106995
Epoch 4 	| Training Accuracy 0.968999981880188
Epoch 5 	| Training Accuracy 0.9789999723434448
Epoch 6 	| Training Accuracy 0.9729999899864197
Epoch 7 	| Training Accuracy 0.9800000190734863
Epoch 8 	| Training Accuracy 0.9729999899864197
Epoch 9 	| Training Accuracy 0.9810000061988831
Epoch 10 	| Training Accuracy 0.9860000014305115
Epoch 11 	| Training Accuracy 0.9819999933242798
Epoch 12 	| Training Accuracy 0.984000027179718
Epoch 13 	| Training Accuracy 0.9860000014305115
Epoch 14 	| Training Accuracy 0.9890000224113464
Epoch 15 	| Training Accuracy 0.9890000224113464
Epoch 16 	| Training Accuracy 0.9890000224113464
Epoch 17 	| Training Accuracy 0.9879999756813049
Epoch 18 	| Training Accuracy 0.9890000224113464
Epoch 19 	| Training Accuracy 0.9890000224113464
Epoch 20 	| Training Accuracy 0.

# Saving the Model

In [46]:
outputs = {'accuracy_calc/prediction': model_prediction}
inputs = {'X': X}
tf.compat.v1.saved_model.simple_save(sess, 'SavedModel', inputs, outputs)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel\saved_model.pb


# Make a prediction

In [47]:
img = Image.open('MNIST/test_img.png')
img

In [48]:
bw = img.convert('L')

In [49]:
img_array = np.invert(bw)

In [50]:
img_array.shape

(28, 28)

In [51]:
test_img = np.ravel(img_array)

In [52]:
test_img.shape

(784,)

In [53]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X: [test_img]})

In [54]:
print(f'prediction for the test image is {prediction}')

prediction for the test image is [2]


# Testing and Evaluation

In [55]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 97.86%


# Reset for the next Run

In [56]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()

# Code for first part of module

In [57]:
# with tf.name_scope('hidden_1'):

#     initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUT, n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_value=initial_w1, name='w1')

#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='b1')

#     layer1_in = tf.matmul(X, w1) + b1

#     layer1_out = tf.nn.relu(layer1_in)

In [58]:
# with tf.name_scope('hidden_2'):

#     initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2, name='w2')

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='b2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [59]:
# with tf.name_scope('output_layer'):

#     initial_w3 = tf.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3, name='w3')

#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3, name='b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [60]:
# b3.eval(sess)